In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from geopy.point import Point
from geopy.geocoders import *
from geopy.distance import *

detroit_demolition_permits = pd.read_json('detroit_demolition_permits.json')
detroit_blight_violations = pd.read_json('detroit_blight_violations.json')
detroit_311 = pd.read_json('detroit_311.json')
detroit_crime= pd.read_json('detroit_crime.json')

In [2]:
detroit_crime = detroit_crime[detroit_crime.label.notnull()]

In [3]:
detroit_311 = detroit_311[detroit_311.label.notnull()]

In [4]:
detroit_crime.label = detroit_crime.label.apply(lambda x: int(x))

In [5]:
detroit_311.label = detroit_311.label.apply(lambda x: int(x))

In [6]:
set_of_label = set(detroit_demolition_permits.label.tolist() + detroit_blight_violations.label.tolist() +detroit_crime.label.tolist() + detroit_311.label.tolist())

In [7]:
set_of_label = list(set_of_label)
set_of_blighted = set(detroit_demolition_permits.label.tolist())

In [8]:
df = pd.DataFrame(set_of_label,index=set_of_label)

In [9]:
df.columns = [['label']]

In [10]:
df['blighted'] =df.label.apply(lambda x: x in set_of_blighted)
df = df.drop('label',1)

In [11]:
df = pd.concat([df,pd.DataFrame(detroit_311.label.value_counts())],1)
def clear(x):
    if pd.isnull(x):
        return 0
    return int(x)
df.label = df.label.apply(clear)

In [12]:
detroit_311 = detroit_311[['label','rating']].groupby('label',as_index=False).mean()

In [13]:
detroit_311 = detroit_311.set_index(detroit_311.label.as_matrix())
detroit_311 = detroit_311.drop('label',1)

In [14]:
df = df.rename(columns={'label':'311 count'})

In [15]:
df = pd.concat([df,detroit_311],1)

In [16]:
def remove_dollar_sim(x):
    if pd.isnull(x):
        return 0
    return float(x.replace('$',''))

In [17]:
detroit_blight_violations.AdminFee = detroit_blight_violations.AdminFee.apply(remove_dollar_sim)
detroit_blight_violations.CleanUpCost = detroit_blight_violations.CleanUpCost.apply(remove_dollar_sim)
detroit_blight_violations.FineAmt = detroit_blight_violations.FineAmt.apply(remove_dollar_sim)
detroit_blight_violations.JudgmentAmt = detroit_blight_violations.JudgmentAmt.apply(remove_dollar_sim)
detroit_blight_violations.LateFee = detroit_blight_violations.LateFee.apply(remove_dollar_sim)

In [18]:
detroit_blight_violations['Cate0'] = (detroit_blight_violations.ViolationCategory == 0).apply(int)
detroit_blight_violations['Cate1'] = (detroit_blight_violations.ViolationCategory == 1).apply(int)

In [19]:
detroit_blight_violations.ViolationCode = detroit_blight_violations.ViolationCode.apply(lambda x: x.replace('/','-').split('-')[0])

In [20]:
detroit_blight_violations.ViolationCode = detroit_blight_violations.ViolationCode.apply(int)

In [21]:
detroit_blight_violations['Code9']  = (detroit_blight_violations.ViolationCode == 9).apply(int)
detroit_blight_violations['Code22']  = (detroit_blight_violations.ViolationCode == 22).apply(int)
detroit_blight_violations['Code61']  = (detroit_blight_violations.ViolationCode == 61).apply(int)

In [22]:
detroit_blight_violations = detroit_blight_violations[['label','AdminFee','CleanUpCost','FineAmt','JudgmentAmt','LateFee','Cate0','Cate1','Code9','Code22','Code61']]

In [23]:
detroit_blight_violations = detroit_blight_violations.groupby('label',as_index = False).sum()

In [24]:
detroit_blight_violations = detroit_blight_violations.set_index(detroit_blight_violations.label.as_matrix())
detroit_blight_violations = detroit_blight_violations.drop('label',1)

In [25]:
df = pd.concat([df,detroit_blight_violations],1)

In [26]:
def clean_df(df):
    df = df.apply(lambda x: x.apply(lambda y: 0 if pd.isnull(y) else y))
    df.to_csv('data.csv',index=False, float_format='%.g')
    return df

In [27]:
detroit_crime[u'CATEGORY'].value_counts().index

Index([u'TRAFFIC VIOLATIONS-MOTORCYCLE VIOLATIONS', u'ASSAULT', u'LARCENY',
       u'DAMAGE TO PROPERTY', u'AGGRAVATED ASSAULT', u'BURGLARY',
       u'STOLEN VEHICLE', u'TRAFFIC VIOLATIONS-DRIVING ON SUSPENDED', u'FRAUD',
       u'ROBBERY', u'DANGEROUS DRUGS', u'OBSTRUCTING JUDICIARY',
       u'WEAPONS OFFENSES', u'ESCAPE', u'SOLICITATION', u'BRIBERY', u'ARSON',
       u'OUIL DISPOSE OF VEHICLE TO AVOID FORFEITURE', u'RUNAWAY',
       u'STOLEN PROPERTY', u'OBSTRUCTING THE POLICE', u'FAMILY OFFENSE',
       u'HOMICIDE', u'OTHER BURGLARY', u'HEALTH-SAFETY', u'KIDNAPING',
       u'FORGERY', u'EXTORTION', u'PUBLIC PEACE', u'ENVIRONMENT',
       u'VAGRANCY (OTHER)', u'LIQUOR', u'CONSPIRACY BY COMPUTER',
       u'EMBEZZLEMENT', u'OUIL', u'REVOKED', u'ANTITRUST', u'TAX REVENUE',
       u'IMMIGRATION', u'NEGLIGENT HOMICIDE', u'OBSCENITY', u'ELECTION LAWS',
       u'JUSTIFIABLE HOMICIDE', u'GAMBLING', u'DRUNKENNESS', u'CONGRESS',
       u'FELONY DEATH FROM FLEEING VEHICLE', u'SOVEREIGNTY', u'MI

In [28]:
crimes = [u'ASSAULT', u'LARCENY',u'DAMAGE TO PROPERTY', u'AGGRAVATED ASSAULT', u'BURGLARY',u'STOLEN VEHICLE', u'FRAUD',
          u'ROBBERY', u'DANGEROUS DRUGS',u'WEAPONS OFFENSES', u'ARSON', u'STOLEN PROPERTY', u'HOMICIDE', u'OTHER BURGLARY', 
          u'HEALTH-SAFETY', u'KIDNAPING',u'FORGERY', u'EXTORTION', u'PUBLIC PEACE',u'VAGRANCY (OTHER)', u'SOVEREIGNTY']

In [29]:
crimes = set(crimes)

In [30]:
detroit_crime['crime'] = detroit_crime.CATEGORY.apply(lambda x: x in crimes).apply(int)

In [31]:
detroit_crime = detroit_crime[['label','crime']].groupby('label',as_index=False).sum()

In [32]:
detroit_crime = detroit_crime.set_index(detroit_crime.label.as_matrix())

In [33]:
detroit_crime =detroit_crime.drop('label',1)

In [34]:
df = pd.concat([df,detroit_crime],1)

In [36]:
clean_df(df)

,blighted,311 count,rating,AdminFee,CleanUpCost,FineAmt,JudgmentAmt,LateFee,Cate0,Cate1,Code9,Code22,Code61,crime
0,True,0,0.000000,160.0,0.0,1800.0,2220.0,180.0,8.0,0.0,4.0,3.0,1.0,0.0
1,True,0,0.000000,1100.0,0.0,11500.0,14030.0,1150.0,55.0,0.0,39.0,16.0,0.0,8.0
2,True,0,0.000000,420.0,0.0,4400.0,5470.0,440.0,21.0,0.0,16.0,4.0,1.0,0.0
3,True,1,2.000000,520.0,0.0,6900.0,8370.0,690.0,26.0,0.0,23.0,3.0,0.0,3.0
4,True,9,3.444444,940.0,0.0,18400.0,21620.0,1840.0,45.0,2.0,38.0,8.0,1.0,7.0
5,True,0,0.000000,180.0,0.0,2900.0,3460.0,290.0,9.0,0.0,6.0,3.0,0.0,0.0
6,True,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,True,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,True,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,True,4,2.500000,7860.0,0.0,121130.0,144390.0,12080.0,393.0,0.0,284.0,109.0,0.0,132.0
